# Reddit Marketing AI Agent - Complete Example Workflow

This notebook demonstrates the complete workflow of the Reddit Marketing AI Agent for a single organization.

## Prerequisites
1. Ensure your `.env` file is configured with:
   - `OPENAI_API_KEY`
   - `GOOGLE_API_KEY`
   - `GROQ_API_KEY` (optional)
   - `FIRECRAWL_API_KEY` (optional)
2. Fill in your Reddit API credentials in Cell 2
3. Run cells in order for the complete workflow

Each cell can be run independently after the initial setup.

In [1]:
# Cell 1: Initial Setup and Imports
import os
import uuid
import logging
from datetime import datetime

# Set up logging to see service output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import models
from app.models.campaign import (
    CampaignCreateRequest, ResponseTone, SubredditDiscoveryRequest,
    PostDiscoveryRequest, ResponseGenerationRequest, ResponseExecutionRequest
)
from app.models.document import DocumentCreateRequest, DocumentIngestURLRequest

# Import settings
from app.core.settings import settings

# Import services
from app.services.campaign_service import CampaignService
from app.services.document_service import DocumentService
from app.services.reddit_service import RedditService
from app.services.llm_service import LLMService
from app.services.analytics_service import AnalyticsService
from app.services.scraper_service import WebScraperService

# Import managers and storage
from app.managers.campaign_manager import CampaignManager
from app.managers.document_manager import DocumentManager
from app.managers.embeddings_manager import EmbeddingsManager
from app.managers.analytics_manager import AnalyticsManager
from app.storage.json_storage import JsonStorage
from app.storage.vector_storage import VectorStorage

# Import clients
from app.clients.llm_client import LLMClient
from app.clients.reddit_client import RedditClient
from app.clients.storage_client import VectorStorageClient

print("✅ All imports successful!")
print(f"📁 Data directory: {settings.DATA_DIR}")
print(f"🤖 Default model: {settings.MODEL_NAME}")

✅ All imports successful!
📁 Data directory: data
🤖 Default model: gpt-4o


In [2]:
# Cell 2: Configuration and Credentials

# Generate unique IDs for this workflow
ORGANIZATION_ID = f"example-org"
CAMPAIGN_NAME = f"Example Campaign"

print(f"🏢 Organization ID: {ORGANIZATION_ID}")
print(f"📋 Campaign Name: {CAMPAIGN_NAME}")

# Reddit API Credentials - FILL THESE IN WITH YOUR ACTUAL CREDENTIALS
REDDIT_CREDENTIALS = {
    "client_id": os.getenv("REDDIT_CLIENT_ID"),
    "client_secret": os.getenv("REDDIT_CLIENT_SECRET"),
    "username": os.getenv("REDDIT_USERNAME"),
    "password": os.getenv("REDDIT_PASSWORD")
}

# Validate environment variables
required_env_vars = ["OPENAI_API_KEY", "GOOGLE_API_KEY"]
missing_vars = [var for var in required_env_vars if not getattr(settings, var, None)]

if missing_vars:
    print(f"❌ Missing required environment variables: {missing_vars}")
    print("Please configure your .env file with the required API keys.")
else:
    print("✅ Environment variables configured")

# Check Reddit credentials
if REDDIT_CREDENTIALS["client_id"] == "YOUR_REDDIT_CLIENT_ID":
    print("⚠️  Please update REDDIT_CREDENTIALS with your actual Reddit API credentials")
    print("   You can get these from: https://www.reddit.com/prefs/apps")
else:
    print("✅ Reddit credentials configured")

🏢 Organization ID: example-org
📋 Campaign Name: Example Campaign
✅ Environment variables configured
✅ Reddit credentials configured


In [3]:
# Cell 3: Initialize Services and Dependencies

print("🔧 Initializing core components...")

# Initialize core components
json_storage = JsonStorage()
vector_storage_client = VectorStorageClient()
reddit_client = RedditClient(
    client_id=REDDIT_CREDENTIALS["client_id"],
    client_secret=REDDIT_CREDENTIALS["client_secret"],
    username=REDDIT_CREDENTIALS["username"],
    password=REDDIT_CREDENTIALS["password"]
)
llm_client = LLMClient()
web_scraper_service = WebScraperService()

print("🔧 Initializing managers...")

# Initialize managers
campaign_manager = CampaignManager(json_storage=json_storage)
document_manager = DocumentManager(json_storage=json_storage)
embeddings_manager = EmbeddingsManager(vector_storage_client=vector_storage_client)
analytics_manager = AnalyticsManager(
    campaign_manager=campaign_manager,
    document_manager=document_manager
)

print("🔧 Initializing services...")

# Initialize services
llm_service = LLMService(llm_client=llm_client)
reddit_service = RedditService(json_storage=json_storage, reddit_client=reddit_client)
vector_storage = VectorStorage(vector_storage_client=vector_storage_client)
document_service = DocumentService(
    document_manager=document_manager,
    vector_storage=vector_storage,
    web_scraper_service=web_scraper_service
)
campaign_service = CampaignService(
    campaign_manager=campaign_manager,
    document_service=document_service,
    reddit_service=reddit_service,
    llm_service=llm_service
)
analytics_service = AnalyticsService(analytics_manager=analytics_manager)

print("✅ All services initialized successfully!")
print("🚀 Ready to start the workflow")

🔧 Initializing core components...
🔧 Initializing managers...
🔧 Initializing services...
✅ All services initialized successfully!
🚀 Ready to start the workflow


In [4]:
# Cell 4: Set up Organization

print(f"🏢 Setting up organization: {ORGANIZATION_ID}")

# Create or get organization
organization = document_service.get_or_create_organization(
    org_id=ORGANIZATION_ID,
    org_name=f"Example Organization {datetime.now().strftime('%Y-%m-%d')}"
)

print(f"✅ Organization created/retrieved:")
print(f"   ID: {organization.id}")
print(f"   Name: {organization.name}")
print(f"   Description: {organization.description}")
print(f"   Documents: {organization.documents_count}")
print(f"   Created: {organization.created_at}")

🏢 Setting up organization: example-org
✅ Organization created/retrieved:
   ID: example-org
   Name: Example Organization 2025-06-21
   Description: Auto-created organization for example-org
   Documents: 0
   Created: 2025-06-21 01:18:57.185311+00:00


In [5]:
# Cell 5: Ingest Document - Direct Content

print("📄 Ingesting document with direct content...")

# Sample content about Python programming
sample_content = """
Python Programming Best Practices and Tips

Python is a versatile and powerful programming language that's perfect for beginners and experts alike. 
Here are some essential tips and best practices for Python development:

1. Code Readability: Python emphasizes readable code. Use meaningful variable names, proper indentation, 
   and follow PEP 8 style guidelines.

2. Virtual Environments: Always use virtual environments to manage dependencies and avoid conflicts 
   between different projects.

3. Error Handling: Use try-except blocks to handle exceptions gracefully. This makes your code more 
   robust and user-friendly.

4. List Comprehensions: Use list comprehensions for creating lists in a more Pythonic way. 
   They're often more readable and efficient than traditional loops.

5. Documentation: Write docstrings for your functions and classes. Good documentation helps other 
   developers (and future you) understand your code.

6. Testing: Write unit tests for your code using frameworks like pytest or unittest. 
   Testing ensures your code works as expected and helps catch bugs early.

7. Package Management: Use pip and requirements.txt to manage your project dependencies. 
   This makes it easy for others to install and run your code.

8. Code Organization: Structure your projects with proper modules and packages. 
   This makes your code more maintainable and reusable.

Remember, the key to becoming a better Python programmer is practice and continuous learning. 
Join Python communities, contribute to open source projects, and keep experimenting with new features and libraries.
"""

# Create document request
doc_request = DocumentCreateRequest(
    title="Python Programming Best Practices",
    content=sample_content,
    metadata={
        "category": "programming",
        "language": "python",
        "type": "tutorial",
        "source": "direct_input"
    }
)

# Ingest the document
success, message, document_ids = document_service.ingest_documents(
    documents=[doc_request.model_dump()],
    org_id=ORGANIZATION_ID
)

if success:
    direct_content_doc_id = document_ids[0]
    print(f"✅ {message}")
    print(f"   Document ID: {direct_content_doc_id}")
else:
    print(f"❌ {message}")
    direct_content_doc_id = None

📄 Ingesting document with direct content...


Calculating embeddings: 0it [00:00, ?it/s]2025-06-21 06:48:58,638 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Calculating embeddings: 1it [00:01,  1.17s/it]
2025-06-21 06:48:58,926 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-06-21 06:48:59,221 - app.clients.storage_client - INFO - Stored 1 documents for org example-org
2025-06-21 06:48:59,221 - app.storage.vector_storage - INFO - Stored 1 chunks for document e1937129-43ec-46d4-9a0f-27deb0ba2471
2025-06-21 06:48:59,237 - app.services.document_service - INFO - Ingested 1 documents (1 chunks) for org example-org


✅ Successfully ingested 1 documents (1 chunks)
   Document ID: e1937129-43ec-46d4-9a0f-27deb0ba2471


In [6]:
# Cell 6: Ingest Document - Simulated File Upload

print("📁 Ingesting document from simulated file upload...")

# Simulate file content about web development
file_content = """
Modern Web Development with Python

Web development with Python has become increasingly popular due to powerful frameworks like Django, 
Flask, and FastAPI. Here's a comprehensive guide to modern Python web development:

Frameworks Overview:
- Django: A high-level framework that encourages rapid development and clean design. Perfect for 
  large applications with built-in admin interface, ORM, and authentication.
- Flask: A lightweight and flexible micro-framework that gives you more control over components. 
  Great for smaller applications and APIs.
- FastAPI: A modern framework for building APIs with automatic documentation and type hints. 
  Excellent performance and developer experience.

Key Concepts:
1. MVC Architecture: Understand Model-View-Controller pattern for organizing your code.
2. RESTful APIs: Design clean and intuitive APIs following REST principles.
3. Database Integration: Use ORMs like SQLAlchemy or Django ORM for database operations.
4. Authentication & Authorization: Implement secure user authentication and permission systems.
5. Frontend Integration: Connect your Python backend with modern frontend frameworks.

Best Practices:
- Use environment variables for configuration
- Implement proper error handling and logging
- Write comprehensive tests for your endpoints
- Use database migrations for schema changes
- Implement caching for better performance
- Follow security best practices (HTTPS, input validation, etc.)

Deployment:
Modern deployment options include Docker containers, cloud platforms like AWS, Google Cloud, 
or Heroku, and serverless functions for specific use cases.
"""

# Create document request for file upload simulation
file_doc_request = DocumentCreateRequest(
    title="Modern Web Development with Python",
    content=file_content,
    metadata={
        "category": "web_development",
        "language": "python",
        "type": "guide",
        "source": "file_upload",
        "filename": "web_development_guide.txt"
    }
)

# Ingest the document
success, message, document_ids = document_service.ingest_documents(
    documents=[file_doc_request.model_dump()],
    org_id=ORGANIZATION_ID
)

if success:
    file_upload_doc_id = document_ids[0]
    print(f"✅ {message}")
    print(f"   Document ID: {file_upload_doc_id}")
else:
    print(f"❌ {message}")
    file_upload_doc_id = None

📁 Ingesting document from simulated file upload...


Calculating embeddings: 0it [00:00, ?it/s]2025-06-21 06:48:59,938 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Calculating embeddings: 1it [00:00,  1.43it/s]
2025-06-21 06:48:59,970 - app.clients.storage_client - INFO - Stored 1 documents for org example-org
2025-06-21 06:48:59,970 - app.storage.vector_storage - INFO - Stored 1 chunks for document 3d91be1f-33be-4ba6-b053-29ce43edd06d
2025-06-21 06:48:59,986 - app.services.document_service - INFO - Ingested 1 documents (1 chunks) for org example-org


✅ Successfully ingested 1 documents (1 chunks)
   Document ID: 3d91be1f-33be-4ba6-b053-29ce43edd06d


In [7]:
# Cell 7: Ingest Document - URL Scraping

print("🌐 Ingesting document from URL scraping...")

# Example URL - using a public article about Python
# Note: Replace with a real URL that you want to scrape
EXAMPLE_URL = "https://realpython.com/python-basics/"

# Create URL ingestion request
url_request = DocumentIngestURLRequest(
    url=EXAMPLE_URL,
    title="Python Basics from Real Python",
    organization_id=ORGANIZATION_ID,
    scraping_method="auto",  # Will try Firecrawl first, then fallback to requests
    chunk_size=1000,
    chunk_overlap=200
)

# Attempt to ingest from URL
try:
    success, message, url_doc_id = await document_service.ingest_document_from_url(
        url=url_request.url,
        organization_id=url_request.organization_id,
        title=url_request.title,
        chunk_size=url_request.chunk_size,
        chunk_overlap=url_request.chunk_overlap,
        scraping_method=url_request.scraping_method
    )
    
    if success:
        print(f"✅ {message}")
        print(f"   Document ID: {url_doc_id}")
        print(f"   Source URL: {EXAMPLE_URL}")
    else:
        print(f"❌ URL scraping failed: {message}")
        print("   This might be due to missing API keys or network issues")
        url_doc_id = None
        
except Exception as e:
    print(f"❌ Error during URL scraping: {str(e)}")
    print("   Continuing with other documents...")
    url_doc_id = None

2025-06-21 06:49:00,015 - app.services.document_service - INFO - Starting URL ingestion for https://realpython.com/python-basics/


🌐 Ingesting document from URL scraping...


2025-06-21 06:49:02,409 - app.services.scraper_service - INFO - Successfully scraped https://realpython.com/python-basics/ with Firecrawl
Calculating embeddings: 0it [00:00, ?it/s]2025-06-21 06:49:03,016 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Calculating embeddings: 1it [00:01,  1.38s/it]
2025-06-21 06:49:03,892 - app.clients.storage_client - INFO - Stored 6 documents for org example-org
2025-06-21 06:49:03,893 - app.storage.vector_storage - INFO - Stored 6 chunks for document 0a85770c-8adf-4cde-a3d3-828abfda4c46
2025-06-21 06:49:03,905 - app.services.document_service - INFO - Ingested 1 documents (6 chunks) for org example-org
2025-06-21 06:49:03,906 - app.services.document_service - INFO - Successfully ingested document from URL https://realpython.com/python-basics/ with ID 0a85770c-8adf-4cde-a3d3-828abfda4c46


✅ Successfully ingested document from URL: https://realpython.com/python-basics/
   Document ID: 0a85770c-8adf-4cde-a3d3-828abfda4c46
   Source URL: https://realpython.com/python-basics/


In [8]:
# Cell 8: Create Campaign

print(f"📋 Creating campaign: {CAMPAIGN_NAME}")

# Create campaign request
campaign_request = CampaignCreateRequest(
    name=CAMPAIGN_NAME,
    description="Example campaign demonstrating Python programming expertise and web development knowledge",
    response_tone=ResponseTone.HELPFUL,
    max_responses_per_day=5
)

# Create the campaign
success, message, campaign = await campaign_service.create_campaign(
    organization_id=ORGANIZATION_ID,
    request=campaign_request
)

if success:
    CAMPAIGN_ID = campaign.id
    print(f"✅ {message}")
    print(f"   Campaign ID: {CAMPAIGN_ID}")
    print(f"   Status: {campaign.status}")
    print(f"   Response Tone: {campaign.response_tone}")
    print(f"   Max Responses/Day: {campaign.max_responses_per_day}")
    print(f"   Created: {campaign.created_at}")
else:
    print(f"❌ {message}")
    CAMPAIGN_ID = None

2025-06-21 06:49:03,921 - app.services.campaign_service - INFO - Created campaign 'Example Campaign' for org example-org


📋 Creating campaign: Example Campaign
✅ Campaign 'Example Campaign' created successfully
   Campaign ID: 022b52d5-4fc3-481d-9eef-fd8618709b5a
   Status: CampaignStatus.CREATED
   Response Tone: ResponseTone.HELPFUL
   Max Responses/Day: 5
   Created: 2025-06-21 01:19:03.916494+00:00


In [ ]:
# Cell 9: Discover Topics and Subreddits

if CAMPAIGN_ID:
    print("🔍 Discovering topics and subreddits...")
    
    # Collect all successfully ingested document IDs
    document_ids = []
    if direct_content_doc_id:
        document_ids.append(direct_content_doc_id)
    if file_upload_doc_id:
        document_ids.append(file_upload_doc_id)
    if url_doc_id:
        document_ids.append(url_doc_id)
    
    if not document_ids:
        print("❌ No documents available for subreddit discovery")
    else:
        print(f"📄 Using {len(document_ids)} documents for discovery")
        
        # Create subreddit discovery request
        subreddit_request = SubredditDiscoveryRequest(
            document_ids=document_ids
        )
        
        # Discover subreddits
        campaign, campaign_context, topics = await campaign_service.discover_topics(
            campaign_id=CAMPAIGN_ID,
            request=subreddit_request
        )

        success, message, discovery_data = await campaign_service.discover_subreddits(
            campaign, campaign_context, topics

        )
        
        if success:
            print(f"✅ {message}")
            print(f"   Topics found: {discovery_data.get('topics', [])}")
            print(f"   Subreddits discovered: {discovery_data.get('subreddits', [])}")
            print(f"   Total subreddits: {discovery_data.get('total_found', 0)}")
            
            # Get updated campaign
            success, _, updated_campaign = await campaign_service.get_campaign(CAMPAIGN_ID)
            if success:
                campaign = updated_campaign
                print(f"   Campaign status: {campaign.status}")
        else:
            print(f"❌ {message}")
else:
    print("❌ Cannot discover subreddits - no campaign created")

2025-06-21 06:49:03,964 - app.services.document_service - INFO - Campaign context prepared: 11860 characters from 3 documents


🔍 Discovering topics and subreddits...
📄 Using 3 documents for discovery


2025-06-21 06:49:04,179 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 06:49:05,668 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-21 06:49:05,705 - google_genai.models - INFO - AFC remote call 1 is done.
2025-06-21 06:49:05,711 - app.services.llm_service - INFO - Extracted 10 topics from content
2025-06-21 06:49:05,713 - app.clients.reddit_client - INFO - Initialized Reddit API client with user: lonlionli
2025-06-21 06:54:10,575 - app.clients.reddit_client - WARNING - Error: received 403 HTTP response. Retrying in 2.00 seconds...
2025-06-21 06:55:09,375 - app.clients.reddit_client - WARNING - Error: received 403 HTTP response. Retrying in 4.00 seconds...
2025-06-21 06:55:14,195 - app.clients.reddit_client - WARNING - Error: received 403 HTTP response. Retrying in 8.00 seconds...
2025-06-21 06:55:22,690 - app.clients.reddit_client - ERROR - Max re

✅ Discovered 10 relevant subreddits
   Topics found: ['Python programming', 'Web development with Python', 'Django framework', 'Flask framework', 'FastAPI framework', 'Python best practices', 'Python tutorials', 'Python courses', 'Python for beginners', 'Python communities']
   Subreddits discovered: ['Python', 'learnpython', 'PythonLearning', 'PythonProjects2', 'pythoncoding', 'django', 'djangolearning', 'flask', 'FastAPI', 'ProgrammingLanguages']
   Total subreddits: 10
   Campaign status: CampaignStatus.SUBREDDITS_DISCOVERED


In [ ]:

if CAMPAIGN_ID:
    print("🔍 Discovering topics and subreddits...")
       
        success, message, discovery_data = await campaign_service.discover_subreddits(
            campaign, campaign_context, topics

        )
        
        if success:
            print(f"✅ {message}")
            print(f"   Topics found: {discovery_data.get('topics', [])}")
            print(f"   Subreddits discovered: {discovery_data.get('subreddits', [])}")
            print(f"   Total subreddits: {discovery_data.get('total_found', 0)}")
            
            # Get updated campaign
            success, _, updated_campaign = await campaign_service.get_campaign(CAMPAIGN_ID)
            if success:
                campaign = updated_campaign
                print(f"   Campaign status: {campaign.status}")
        else:
            print(f"❌ {message}")
else:
    print("❌ Cannot discover subreddits - no campaign created")

In [10]:
# Cell 10: Discover Posts

if CAMPAIGN_ID and campaign and campaign.target_subreddits:
    print("📝 Discovering relevant posts...")
    
    # Use discovered subreddits (limit to first 3 for demo)
    target_subreddits = campaign.target_subreddits[:3]
    print(f"🎯 Targeting subreddits: {target_subreddits}")
    
    # Create post discovery request
    post_request = PostDiscoveryRequest(
        subreddits=target_subreddits,
        max_posts_per_subreddit=5,  # Limit for demo
        time_filter="day",
        reddit_credentials=REDDIT_CREDENTIALS
    )
    
    # Discover posts
    try:
        success, message, post_data = await campaign_service.discover_posts(
            campaign_id=CAMPAIGN_ID,
            request=post_request
        )
        
        if success:
            print(f"✅ {message}")
            print(f"   Posts found: {post_data.get('posts_found', 0)}")
            print(f"   Subreddits searched: {post_data.get('subreddits_searched', 0)}")
            
            # Show sample posts
            posts = post_data.get('posts', [])
            if posts:
                print("\n📋 Sample discovered posts:")
                for i, post in enumerate(posts[:3]):  # Show first 3
                    print(f"   {i+1}. {post.get('title', 'No title')[:60]}...")
                    print(f"      Subreddit: r/{post.get('subreddit', 'unknown')}")
                    print(f"      Score: {post.get('score', 0)}")
                    print(f"      Relevance: {post.get('relevance_score', 0):.2f}")
            
            # Get updated campaign
            success, _, updated_campaign = await campaign_service.get_campaign(CAMPAIGN_ID)
            if success:
                campaign = updated_campaign
                print(f"\n   Campaign status: {campaign.status}")
        else:
            print(f"❌ {message}")
            
    except Exception as e:
        print(f"❌ Error discovering posts: {str(e)}")
        print("   This might be due to Reddit API credentials or rate limiting")
        
else:
    print("❌ Cannot discover posts - no campaign or subreddits available")
    if CAMPAIGN_ID:
        print("   Make sure subreddit discovery completed successfully")

2025-06-21 06:55:24,320 - app.services.document_service - INFO - Campaign context prepared: 11860 characters from 3 documents
2025-06-21 06:55:24,321 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.


📝 Discovering relevant posts...
🎯 Targeting subreddits: ['Python', 'learnpython', 'PythonLearning']


2025-06-21 06:55:25,439 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-21 06:55:25,488 - google_genai.models - INFO - AFC remote call 1 is done.
2025-06-21 06:55:25,488 - app.services.llm_service - INFO - Extracted 10 topics from content
2025-06-21 06:55:25,495 - app.clients.reddit_client - INFO - Initialized Reddit API client with user: lonlionli
2025-06-21 06:56:23,156 - app.clients.reddit_client - INFO - Closed Reddit API client
2025-06-21 06:56:23,157 - app.services.reddit_service - INFO - Discovered 37 posts across 3 subreddits
2025-06-21 06:56:23,157 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 06:56:24,662 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-21 06:56:24,663 - google_genai.models - INFO - AFC remote call 1 is done.
2025-06-21 

✅ Found 37 relevant posts
   Posts found: 37
   Subreddits searched: 3

📋 Sample discovered posts:
   1. Saturday Daily Thread: Resource Request and Sharing! Daily T...
      Subreddit: r/Python
      Score: 3
      Relevance: 0.90
   2. package-ui.nvim now supports pip/python...
      Subreddit: r/Python
      Score: 6
      Relevance: 0.80
   3. Mom Java is eating my AI lunch...
      Subreddit: r/Python
      Score: 0
      Relevance: 0.70

   Campaign status: CampaignStatus.POSTS_FOUND


In [11]:
# Cell 11: Generate Responses

if CAMPAIGN_ID and campaign and campaign.target_posts:
    print("💬 Generating responses for target posts...")
    
    # Select first few posts for response generation (limit for demo)
    target_post_ids = list(campaign.target_posts.keys())[:3]
    print(f"🎯 Generating responses for {len(target_post_ids)} posts")
    
    # Create response generation request
    response_request = ResponseGenerationRequest(
        target_post_ids=target_post_ids,
        tone=ResponseTone.HELPFUL  # Override campaign tone if needed
    )
    
    # Generate responses
    try:
        success, message, response_data = await campaign_service.generate_responses(
            campaign_id=CAMPAIGN_ID,
            request=response_request
        )
        
        if success:
            print(f"✅ {message}")
            print(f"   Responses generated: {response_data.get('responses_generated', 0)}")
            
            # Show sample responses
            responses = response_data.get('responses', [])
            if responses:
                print("\n💬 Sample generated responses:")
                for i, response in enumerate(responses[:2]):  # Show first 2
                    print(f"\n   Response {i+1}:")
                    print(f"   Target Post: {response.get('target_post_id', 'unknown')[:20]}...")
                    print(f"   Confidence: {response.get('confidence_score', 0):.2f}")
                    print(f"   Content Preview: {response.get('response_content', '')[:100]}...")
            
            # Get updated campaign
            success, _, updated_campaign = await campaign_service.get_campaign(CAMPAIGN_ID)
            if success:
                campaign = updated_campaign
                print(f"\n   Campaign status: {campaign.status}")
        else:
            print(f"❌ {message}")
            
    except Exception as e:
        print(f"❌ Error generating responses: {str(e)}")
        print("   This might be due to LLM API issues or rate limiting")
        
else:
    print("❌ Cannot generate responses - no campaign or target posts available")
    if CAMPAIGN_ID:
        print("   Make sure post discovery completed successfully")

2025-06-21 06:57:13,157 - app.services.document_service - INFO - Campaign context prepared: 11860 characters from 3 documents
2025-06-21 06:57:13,159 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.


💬 Generating responses for target posts...
🎯 Generating responses for 3 posts


2025-06-21 06:57:15,372 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-21 06:57:15,420 - google_genai.models - INFO - AFC remote call 1 is done.
2025-06-21 06:57:15,427 - app.services.llm_service - INFO - Generated response with confidence: 0.90
2025-06-21 06:57:15,427 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 06:57:17,247 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-21 06:57:17,301 - google_genai.models - INFO - AFC remote call 1 is done.
2025-06-21 06:57:17,302 - app.services.llm_service - INFO - Generated response with confidence: 0.90
2025-06-21 06:57:17,303 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 06:57:19,855 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1be

✅ Generated 3 responses
   Responses generated: 3

💬 Sample generated responses:

   Response 1:
   Target Post: 0ef9f540-69b9-4045-9...
   Confidence: 0.90
   Content Preview: Great initiative with this resource sharing thread! For those just starting out and looking for a ro...

   Response 2:
   Target Post: 642b0ebe-327f-42df-8...
   Confidence: 0.90
   Content Preview: This looks like a really useful tool, especially for those working with multiple languages and packa...

   Campaign status: CampaignStatus.RESPONSES_PLANNED


In [12]:
# Cell 12: Post Responses (CAUTION: This will actually post to Reddit!)

# WARNING: This cell will actually post responses to Reddit!
# Only run this if you want to make real posts
ACTUALLY_POST_TO_REDDIT = False  # Set to True to enable actual posting

if CAMPAIGN_ID and campaign and campaign.planned_responses and ACTUALLY_POST_TO_REDDIT:
    print("🚀 Posting responses to Reddit...")
    print("⚠️  WARNING: This will make actual posts to Reddit!")
    
    # Select first few planned responses (limit for demo)
    planned_response_ids = list(campaign.planned_responses.keys())[:2]
    print(f"📤 Posting {len(planned_response_ids)} responses")
    
    # Create response execution request
    execution_request = ResponseExecutionRequest(
        planned_response_ids=planned_response_ids,
        reddit_credentials=REDDIT_CREDENTIALS
    )
    
    # Execute responses
    try:
        success, message, execution_data = await campaign_service.execute_responses(
            campaign_id=CAMPAIGN_ID,
            request=execution_request
        )
        
        if success:
            print(f"✅ {message}")
            print(f"   Responses posted: {execution_data.get('responses_posted', 0)}")
            print(f"   Responses failed: {execution_data.get('responses_failed', 0)}")
            
            # Show posting results
            posted_responses = execution_data.get('posted_responses', [])
            if posted_responses:
                print("\n📤 Posting results:")
                for i, response in enumerate(posted_responses):
                    status = "✅ Success" if response.get('posting_successful') else "❌ Failed"
                    print(f"   {i+1}. {status}")
                    if response.get('posting_successful'):
                        print(f"      Reddit ID: {response.get('reddit_comment_id', 'unknown')}")
                        print(f"      Permalink: {response.get('reddit_permalink', 'unknown')}")
                    else:
                        print(f"      Error: {response.get('error_message', 'Unknown error')}")
            
            # Get updated campaign
            success, _, updated_campaign = await campaign_service.get_campaign(CAMPAIGN_ID)
            if success:
                campaign = updated_campaign
                print(f"\n   Campaign status: {campaign.status}")
        else:
            print(f"❌ {message}")
            
    except Exception as e:
        print(f"❌ Error posting responses: {str(e)}")
        print("   This might be due to Reddit API issues or authentication problems")
        
elif CAMPAIGN_ID and campaign and campaign.planned_responses:
    print("⚠️  Response posting is disabled for safety")
    print(f"   {len(campaign.planned_responses)} responses are ready to post")
    print("   Set ACTUALLY_POST_TO_REDDIT = True to enable actual posting")
    print("   ⚠️  WARNING: This will make real posts to Reddit!")
else:
    print("❌ Cannot post responses - no campaign or planned responses available")
    if CAMPAIGN_ID:
        print("   Make sure response generation completed successfully")

⚠️  Response posting is disabled for safety
   3 responses are ready to post
   Set ACTUALLY_POST_TO_REDDIT = True to enable actual posting
   ⚠️  WARNING: This will make real posts to Reddit!


In [13]:
# Cell 13: Fetch and Display Analytics

if CAMPAIGN_ID:
    print("📊 Fetching analytics and reports...")
    
    # Get organization quick stats
    print("\n🏢 Organization Quick Stats:")
    try:
        quick_stats = analytics_service.get_quick_stats(ORGANIZATION_ID)
        if "error" not in quick_stats:
            print(f"   Total Campaigns: {quick_stats.get('total_campaigns', 0)}")
            print(f"   Active Campaigns: {quick_stats.get('active_campaigns', 0)}")
            print(f"   Total Documents: {quick_stats.get('total_documents', 0)}")
            print(f"   Success Rate: {quick_stats.get('success_rate', 0):.1f}%")
        else:
            print(f"   Error: {quick_stats['error']}")
    except Exception as e:
        print(f"   Error fetching quick stats: {str(e)}")
    
    # Get campaign engagement report
    print("\n📋 Campaign Engagement Report:")
    try:
        engagement_report = analytics_service.get_campaign_engagement_report(CAMPAIGN_ID)
        if "error" not in engagement_report:
            basic_stats = engagement_report.get('basic_stats', {})
            engagement_metrics = engagement_report.get('engagement_metrics', {})
            
            print(f"   Campaign: {basic_stats.get('campaign_name', 'Unknown')}")
            print(f"   Status: {basic_stats.get('status', 'Unknown')}")
            print(f"   Documents Selected: {basic_stats.get('documents_selected', 0)}")
            print(f"   Subreddits Found: {basic_stats.get('subreddits_found', 0)}")
            print(f"   Posts Found: {basic_stats.get('posts_found', 0)}")
            print(f"   Responses Planned: {basic_stats.get('responses_planned', 0)}")
            print(f"   Responses Posted: {basic_stats.get('responses_posted', 0)}")
            print(f"   Successful Posts: {basic_stats.get('successful_posts', 0)}")
            print(f"   Failed Posts: {basic_stats.get('failed_posts', 0)}")
            
            if engagement_metrics:
                print(f"   Engagement Rate: {engagement_metrics.get('engagement_rate', 0):.1f}%")
                print(f"   Success Rate: {engagement_metrics.get('success_rate', 0):.1f}%")
        else:
            print(f"   Error: {engagement_report['error']}")
    except Exception as e:
        print(f"   Error fetching engagement report: {str(e)}")
    
    # Get subreddit effectiveness report
    print("\n🎯 Subreddit Effectiveness Report:")
    try:
        subreddit_report = analytics_service.get_subreddit_effectiveness_report(ORGANIZATION_ID)
        if "error" not in subreddit_report:
            ranked_subreddits = subreddit_report.get('ranked_subreddits', [])
            recommendations = subreddit_report.get('recommendations', [])
            
            print(f"   Total Subreddits Analyzed: {subreddit_report.get('total_subreddits_analyzed', 0)}")
            
            if ranked_subreddits:
                print("   Top Performing Subreddits:")
                for i, subreddit in enumerate(ranked_subreddits[:3]):
                    name = subreddit.get('subreddit', 'unknown')
                    score = subreddit.get('effectiveness_score', 0)
                    stats = subreddit.get('stats', {})
                    print(f"     {i+1}. r/{name} (Score: {score:.1f})")
                    print(f"        Posts Targeted: {stats.get('posts_targeted', 0)}")
                    print(f"        Engagement Rate: {stats.get('engagement_rate', 0):.1f}%")
            
            if recommendations:
                print("   Recommendations:")
                for rec in recommendations:
                    print(f"     • {rec}")
        else:
            print(f"   Error: {subreddit_report['error']}")
    except Exception as e:
        print(f"   Error fetching subreddit report: {str(e)}")
    
    # Get organization performance report
    print("\n📈 Organization Performance Report:")
    try:
        performance_report = analytics_service.get_organization_performance_report(ORGANIZATION_ID)
        if "error" not in performance_report:
            insights = performance_report.get('performance_insights', [])
            
            if insights:
                print("   Performance Insights:")
                for insight in insights:
                    print(f"     • {insight}")
            else:
                print("   No specific insights available yet")
                print("   (More data needed for detailed analysis)")
        else:
            print(f"   Error: {performance_report['error']}")
    except Exception as e:
        print(f"   Error fetching performance report: {str(e)}")
    
    print("\n✅ Analytics report complete!")
    
else:
    print("❌ Cannot fetch analytics - no campaign available")

📊 Fetching analytics and reports...

🏢 Organization Quick Stats:
   Total Campaigns: 1
   Active Campaigns: 1
   Total Documents: 3
   Success Rate: 0.0%

📋 Campaign Engagement Report:
   Campaign: Example Campaign
   Status: CampaignStatus.RESPONSES_PLANNED
   Documents Selected: 3
   Subreddits Found: 10
   Posts Found: 37
   Responses Planned: 3
   Responses Posted: 0
   Successful Posts: 0
   Failed Posts: 0
   Engagement Rate: 8.1%
   Success Rate: 0.0%

🎯 Subreddit Effectiveness Report:
   Total Subreddits Analyzed: 3
   Top Performing Subreddits:
     1. r/Python (Score: 15.0)
        Posts Targeted: 12
        Engagement Rate: 25.0%
     2. r/learnpython (Score: 0.0)
        Posts Targeted: 16
        Engagement Rate: 0.0%
     3. r/PythonLearning (Score: 0.0)
        Posts Targeted: 9
        Engagement Rate: 0.0%
   Recommendations:
     • r/PythonLearning shows low effectiveness - review targeting strategy

📈 Organization Performance Report:
   Performance Insights:
     • L

In [14]:
# Cell 14: Cleanup and Summary

print("🧹 Cleaning up resources...")

# Cleanup Reddit service
try:
    await reddit_service.cleanup()
    print("✅ Reddit client cleaned up")
except Exception as e:
    print(f"⚠️  Error during Reddit cleanup: {str(e)}")

# Cleanup campaign service
try:
    await campaign_service.cleanup()
    print("✅ Campaign service cleaned up")
except Exception as e:
    print(f"⚠️  Error during campaign cleanup: {str(e)}")

print("\n📋 Workflow Summary:")
print(f"   Organization ID: {ORGANIZATION_ID}")
if CAMPAIGN_ID:
    print(f"   Campaign ID: {CAMPAIGN_ID}")
    print(f"   Campaign Name: {CAMPAIGN_NAME}")
else:
    print("   Campaign: Not created")

# Document ingestion summary
doc_count = 0
if direct_content_doc_id:
    doc_count += 1
    print(f"   ✅ Direct content document: {direct_content_doc_id}")
if file_upload_doc_id:
    doc_count += 1
    print(f"   ✅ File upload document: {file_upload_doc_id}")
if url_doc_id:
    doc_count += 1
    print(f"   ✅ URL scraped document: {url_doc_id}")

print(f"   Total documents ingested: {doc_count}")

# Campaign progress summary
if campaign:
    print(f"   Campaign status: {campaign.status}")
    print(f"   Subreddits discovered: {len(campaign.target_subreddits)}")
    print(f"   Posts found: {len(campaign.target_posts)}")
    print(f"   Responses planned: {len(campaign.planned_responses)}")
    print(f"   Responses posted: {len(campaign.posted_responses)}")

print("\n🎉 Example workflow completed!")
print("\n📝 Notes:")
print("   • Data is stored in the 'data' directory")
print("   • You can run individual cells to repeat specific steps")
print("   • To reset, delete the 'data' directory and restart")
print("   • Check the API documentation at http://localhost:8000/docs when running the server")

🧹 Cleaning up resources...
✅ Reddit client cleaned up
✅ Campaign service cleaned up

📋 Workflow Summary:
   Organization ID: example-org
   Campaign ID: 022b52d5-4fc3-481d-9eef-fd8618709b5a
   Campaign Name: Example Campaign
   ✅ Direct content document: e1937129-43ec-46d4-9a0f-27deb0ba2471
   ✅ File upload document: 3d91be1f-33be-4ba6-b053-29ce43edd06d
   ✅ URL scraped document: 0a85770c-8adf-4cde-a3d3-828abfda4c46
   Total documents ingested: 3
   Campaign status: CampaignStatus.RESPONSES_PLANNED
   Subreddits discovered: 10
   Posts found: 37
   Responses planned: 3
   Responses posted: 0

🎉 Example workflow completed!

📝 Notes:
   • Data is stored in the 'data' directory
   • You can run individual cells to repeat specific steps
   • To reset, delete the 'data' directory and restart
   • Check the API documentation at http://localhost:8000/docs when running the server
